# Merkle Trees

In [1]:
!pip3 install pymerkle

In [2]:
import pymerkle
import json
from hashlib import sha256
import random

Suppose you had a list with some entries and some party wanted to access some entries of the list. The caveat is that they want proof that the entry is definitely from the list and not forged by you. How would you go about doing it? One common way is to have what we call a hash checksum, and we can send the entire list along with the checksum, so that they can access the entry while knowing that it is valid. The question now is, what do we do if the list is large, say containing millions of rows of data? Is it viable to send the entire list and checksum, just so the other person can access a few entries? A solution we can use is merkle trees. Merkle trees are a type of hashing a list, database, or anything that offers a type of commitment. Once the sender hashes the tree, they're commiting to that tree and can't modify it, like a checksum. The main diffrence with a merkle tree is that you can send the checksum, the entry and a proof that the entry is from the checksummed database. We will go over the proof later below
we have an image showing how it works:
![Hash_Tree](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Hash_Tree.svg/500px-Hash_Tree.svg.png)

Here we have data L1 to L4, and to hash the list, we first hash every value, and then further hash every pair of hashes until we are left with a single hash value, which is our root value. With this, we can get a checksum property to show that the list is original

Let's do our own example. Here we have a list with four entries:

In [3]:
list = ["entry1", "entry2", "entry3", "entry4"]

In [4]:
#merkle tree. disabling security so that we can verify the hashes easily
tree = pymerkle.InmemoryTree(algorithm='sha256', disable_security = True)

In [5]:
#adding entries to the merkle tree
for item in list:
    tree.append_entry(bytes(item, 'utf-8'))

In [6]:
print(tree)



 └─66d65d6a...
    ├──f8e12626...
    │   ├──c17dd901...
    │   └──ad206374...
    └──85dd631d...
        ├──a671a481...
        └──81f7edb5...




In the tree above, we have 4 leaves, which are the four entries in the list, where the topmost leaf is the first leaf and the bottom most leaf is the last entry, we can see that the subsequent non-leaf values are the hash of the nodes two children. This is repeated until we reach the root, which is the checksum. We can use the get_leaf(index) to get the hash of each of the entries and get_state() to get the root value.

In [7]:
print("Root Value / Hash of entire list: ", tree.get_state().hex())
print("Hash of entry 1: ", tree.get_leaf(1).hex())

Root Value / Hash of entire list:  66d65d6a53762dc76b9de87d6c0dc692f5a13075ac8d530a4d86999095660f59
Hash of entry 1:  c17dd9010a5c6b0e5b2ad5a845762d8b206e6166a4e63d32deca8c5664fdfcac


In [8]:
#We can see that the first entry hashes to the leaf value correctly
sha256((list[0]).encode('utf-8')).hexdigest()

'c17dd9010a5c6b0e5b2ad5a845762d8b206e6166a4e63d32deca8c5664fdfcac'

So how do we use this tree to verify than an entry hashes to the root? We essentially just send the sibling of the entry at every level. So in the diagram, if we wanted to send L1, we would send L1, the cheksum Top Hash, and the sibling list Hash 0-1 and Hash 1. So to verify, the other party would hash L1 to get Hash 0-0, hash it together with Hash 0-1 to get Hash 0, and then hash it with Hash 1 to get Top Hash and verify if it matches the provided Top Hash.
for the tree we have the proof below

In [9]:
proof = tree.prove_inclusion(1)
proof_json = proof.serialize()
print(json.dumps(proof_json, indent=4))

{
    "metadata": {
        "algorithm": "sha256",
        "security": false,
        "size": 4
    },
    "rule": [
        0,
        0,
        0
    ],
    "subset": [],
    "path": [
        "c17dd9010a5c6b0e5b2ad5a845762d8b206e6166a4e63d32deca8c5664fdfcac",
        "ad2063741cce2d9f2862b07152b06528d175e9e658ade8f2daa416834c9c089a",
        "85dd631d817b68af765949d46f030ba2896c25d68ad884cf871de10b526f14ff"
    ]
}


we can see the path list in the proof contains all the siblings, until the root

In [10]:
#hashing algorithm that hashes two diffrent hashes together
hasher = pymerkle.hasher.MerkleHasher(tree.algorithm, tree.security)

For the verification, the verifier needs the root, the entry value that needs to be verified, and the proof, which is the list of all the siblings

In [11]:
#We look at how a reciever can verify the entry, using only the following three variables:
root = tree.get_state().hex()
entry = list[0]
proof_siblings = proof_json["path"]

#we first verify that the proof provided is for the correct entry by seeing if the entry hashes to the leaf value in the proof
leaf = sha256((entry).encode('utf-8'))
print("Is the proof provided for the correct entry?: ", leaf.hexdigest() == proof_siblings[0])

#we now iteratively hash the leaf with its sibling until we get to the last value and check if it matches the root value, which is the checksum.
curr = leaf.digest()
for i in range(len(proof_siblings)-1):
    curr = hasher.hash_pair(curr,bytes.fromhex(proof_siblings[i+1]))
print("Does the entry hash into the root?: ", root == curr.hex())


Is the proof provided for the correct entry?:  True
Does the entry hash into the root?:  True


Security of the hash function

Now that we showed the commitment part, let's look at the how the merkle tree is smaller. We compare merkle tree against just sending the entire list, which is a trivial way for a reciever to receieve an entry and be sure that the entry they recieved is from the list.

In [12]:
print("Length of each proof: ", len(bytes(json.dumps(proof_json).encode('utf-8'))))
print("Length of array: ", len(bytes(''.join(list), 'utf-8')))

Length of each proof:  314
Length of array:  24


This makes sense since the proof of the hash contains 3 values, and along with the entry and the checksum, we're sending more data to prove one entry than the entire list itself. But note that the size of proof grows with log n, compared to the list which is n. So let's look at a bigger list, say one with 10,000 entries

In [13]:
large_list = []
for i in range(10000):
    large_list.append(''.join(random.choice("abcdefghijklmnopqrstuvwxyz") for _ in range(256)))

In [14]:
tree_2 = pymerkle.InmemoryTree(algorithm='sha256', disable_security = True)
for item in large_list:
    tree_2.append_entry(bytes(item, 'utf-8'))

In [15]:
proof = tree_2.prove_inclusion(30)
proof_json = proof.serialize()
print(json.dumps(proof_json, indent=4))

{
    "metadata": {
        "algorithm": "sha256",
        "security": false,
        "size": 10000
    },
    "rule": [
        1,
        0,
        1,
        1,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0
    ],
    "subset": [],
    "path": [
        "9775c3d2bee5afff4c8916981313632a3bc0a15c1e53cf002271b00302ca560c",
        "f98be43e3f1fbe181b910976396fffb52e0cda43324c4e9b66f7d12763a6a070",
        "55f87db5b892fb6460e6c3f08a5d151a37b8cd5cf24fa3579c92724b116d17ca",
        "e3f62c0d167165508b65e9749319820b9610dc519bbf93cec8e2815e8cf156ee",
        "28ef2b282a6a03bbfa7878fe010808e10361b67b6ca25308d4801ee698a145dc",
        "5faba11bc7b65c9e722f71e3e998c44320aa4879494faa510a44d37aa8903276",
        "d8612972d84b90ee3f0a54f92fbba3447cae43b1672f3ce71c227856a4b1a14f",
        "b692e77915ed7533b8ab94229167dbbca0ef6a97538d46b83d66db00e15bdde3",
        "a9865da6aae74be21ee484a485d384d086d06d8bef30a1ab53227c98a32

In [16]:
print("Length of each proof: ", len(bytes(json.dumps(proof_json).encode('utf-8'))))
print("Length of array: ", len(bytes(''.join(large_list), 'utf-8')))

Length of each proof:  1170
Length of array:  2560000


With larger lists like this, we see how it is better to use merkle tree than just sending the list . Note that the proof length we have is for each entry, so merkle trees are only efficient if the second party is only requesting for a few entries, in this case, having a merkle tree is only more efficient than sending the entire list if the second party requests less than 2000 entries out of the 10,000 size list.

Merkle trees are really useful for this case, and are used alot in cryptocurrency and in theoretical settings to construct SNARGs(this will be a topic for a diffrent notebook)